In [ ]:

import pandas as pd
from ucimlrepo import fetch_ucirepo
from pathlib import Path

# Load UCI dataset (id=45) and persist
heart = fetch_ucirepo(id=45)
X = heart.data.features.copy()
y = heart.data.targets.copy()
y = (y.iloc[:,0] > 0).astype(int).rename("target")
df = pd.concat([X, y], axis=1)
out = Path("../data/heart_disease.csv")
df.to_csv(out, index=False)
df.head()
